# Sentiment classification

In [ ]:
!pip uninstall --yes torch torchvision torchtext torchaudio
!pip install perceiver-io[text]==0.4.0

In [2]:
# Download tokenizers
!wget -nc https://martin-krasser.com/perceiver/tokenizers.zip
!unzip -qo tokenizers.zip

--2022-07-21 13:08:20--  https://martin-krasser.com/perceiver/tokenizers.zip
Resolving martin-krasser.com (martin-krasser.com)... 217.160.0.142, 2001:8d8:100f:f000::209
Connecting to martin-krasser.com (martin-krasser.com)|217.160.0.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191711 (187K) [application/zip]
Saving to: ‘tokenizers.zip’

tokenizers.zip      100%[===================>] 187.22K   561KB/s    in 0.3s    

2022-07-21 13:08:21 (561 KB/s) - ‘tokenizers.zip’ saved [191711/191711]



In [3]:
# Download checkpoints
!wget -nc -O logs.zip https://martin-krasser.com/perceiver/logs-update-5.zip
!unzip -qo logs.zip

--2022-07-21 13:08:35--  https://martin-krasser.com/perceiver/logs-update-5.zip
Resolving martin-krasser.com (martin-krasser.com)... 217.160.0.142, 2001:8d8:100f:f000::209
Connecting to martin-krasser.com (martin-krasser.com)|217.160.0.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128967640 (123M) [application/zip]
Saving to: ‘logs.zip’

logs.zip            100%[===================>] 122.99M  29.3MB/s    in 4.9s    

2022-07-21 13:08:41 (25.3 MB/s) - ‘logs.zip’ saved [128967640/128967640]



In [4]:
import torch

from perceiver.data.text import TextPreprocessor
from perceiver.model.text.classifier import LitTextClassifier

In [5]:
preprocessor = TextPreprocessor(tokenizer='tokenizers/bert-base-uncased-10k-bookcorpus-ext', max_seq_len=512)

In [6]:
ckpt_path = 'logs/clf/version_1/checkpoints/epoch=028-val_loss=0.274.ckpt'

model = LitTextClassifier.load_from_checkpoint(ckpt_path, clf_ckpt=None).model
model.eval();

In [7]:
text_batch = [
    "I've seen this movie yesterday and it was really boring",
    "I can recommend this movie to all fantasy movie lovers"
]

In [8]:
with torch.no_grad():
    logits = model(*preprocessor.preprocess_batch(text_batch))
    preds = logits.argmax(dim=1)

In [9]:
for text, pred in zip(text_batch, preds):
    print(f'{text} (positive = {pred == 1})')

I've seen this movie yesterday and it was really boring (positive = False)
I can recommend this movie to all fantasy movie lovers (positive = True)
